# devlog 2024-06-12

_author: Trevor Johnson_

We have a new class of ADRIO capable of loading in data from CSV files with shapes N, TxN, and NxN. CSV files used must have a column (or two in the NxN case) to identify geographic location and a column containing the relevant data. A time column in YYYY-MM-DD format must also be included if loading in time-series data. Available formats for geographic identifiers are state_abbrev (AZ), county_state, (Maricopa County, Arizona), and geoid (04013).

The following notebook runs through one example of each supported data shape and label type using locally stored CSV files.

In [1]:
from pathlib import Path

from epymorph.data_shape import Shapes
from epymorph.geo.adrio.file.adrio_file import FileSpec, FileSpecTime, FileSpecMatrix
from epymorph.geo.spec import DynamicGeoSpec, Year
from epymorph.geography.us_census import StateScope, CountyScope
from epymorph.simulation import geo_attrib

spec = DynamicGeoSpec(
    attributes=[
        geo_attrib('label', str, Shapes.N),
        geo_attrib('population', int, Shapes.N),
    ],
    time_period=Year(2020),
    scope=StateScope.in_states(['12', '13', '24', '37', '45', '51']),
    source={
        'label': "Census:name",
        'population': FileSpec(file_path=Path("./epymorph/data/geo/csv/pei_population.csv"),
                               key_col=0, data_col=1, key_type="state_abbrev", skiprows=None),
    }
)

In [2]:
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo

geo = DynamicGeo.from_library(spec, adrio_maker_library)

geo.validate()

In [3]:
spec = DynamicGeoSpec(
    attributes=[
        geo_attrib('label', str, Shapes.N),
        geo_attrib('population', int, Shapes.N),
        geo_attrib('population_0-19', int, Shapes.N),
        geo_attrib('population_20-64', int, Shapes.N),
        geo_attrib('population_65+', int, Shapes.N),
    ],
    time_period=Year(2015),
    scope=CountyScope.in_states(['04', '08', '49', '35', '32']),
    source={
        'label': 'Census:name',
        'population': 'Census',
        'population_0-19': FileSpecTime(file_path=Path("./epymorph/data/geo/csv/us_sw_counties_population.csv"),
                                        time_col=0, key_col=1, data_col=2, key_type="county_state", skiprows=1),
        'population_20-64': FileSpecTime(file_path=Path("./epymorph/data/geo/csv/us_sw_counties_population.csv"),
                                         time_col=0, key_col=1, data_col=3, key_type="county_state", skiprows=1),
        'population_65+': FileSpecTime(file_path=Path("./epymorph/data/geo/csv/us_sw_counties_population.csv"),
                                       time_col=0, key_col=1, data_col=4, key_type="county_state", skiprows=1),
    }
)

In [4]:
geo = DynamicGeo.from_library(spec, adrio_maker_library)

geo.validate()

In [5]:
spec = DynamicGeoSpec(
    attributes=[
        geo_attrib('label', str, Shapes.N),
        geo_attrib('population', int, Shapes.N),
        geo_attrib('commuters', int, Shapes.NxN)
    ],
    time_period=Year(2015),
    scope=CountyScope.in_counties(['35001', '04013', '04017']),
    source={
        'label': 'Census:name',
        'population': 'Census',
        'commuters': FileSpecMatrix(file_path=Path("./epymorph/data/geo/csv/counties_commuters.csv"),
                                    from_key_col=0, to_key_col=1, data_col=2, key_type="geoid", skiprows=1)
    }
)

In [6]:
geo = DynamicGeo.from_library(spec, adrio_maker_library)

geo.validate()